# Beyond Promises

## Async Patterns in JavaScript

<br/>
<br/>
<br/>

<center>by Thales Mello</center>

# Let's start with a story

Weird JavaScript behavior had always kept me from sharing any interest with Node.

But in 2015, I was helping friend build a chat microservice for his app

The alternative we chose to solve the problem as a combination of [Node](https://nodejs.org/en/) + [SocketIO](https://socket.io/)...

and it I became really impressed with the final results!

But frankly, the programming environment felt very awkward.

Anyhow, it was enough to stir up my interest about asynchronous programming.

# What exectly do you mean with awkward programming environment?

Callbacks.

In Node, callbacks are the builtin way of invoking asynchronous code, and in the first years of Node, it was the only way.

And because asynchronous programming was the whole point of using Node, I frankly used callbacks for everything.

# Node

- Allows us to create fast servers relying on JavaScript's event driven approach to programming
- But working with async programming can be hard

# How exactly callbacks work?

In [32]:
var sleep = time => {
    let start = new Date().getTime()
    let expire = start + time;

    while (new Date().getTime() < expire);
}

In [33]:
// Consider this sync example. The 'world' is delayed when compared
// to the first line. But it happens at the cost of blocking JavaScript's
// single thread.
console.log('Hello')
sleep(1000)
console.log('world!')

Hello
world!


In [34]:
var progress = fn => {
  var handle = setInterval(() => process.stdout.write('.'), 50)
  fn(() => clearTimeout(handle))
}

progress(done => {
    console.log('Hello')
    sleep(1000)
    console.log('world!')
    done()
})

Hello
world!


In [35]:
var timeout = (time, fn) => setTimeout(fn, time)

progress(done => {
    console.log('Hello')
    timeout(1000, () => {
        console.log('world!')
        done()
    })
})

Hello
..................world!


# Let's work with a two servers example 

In [39]:
progress(done => {
  var queueCallback, serverHandler
  console.log("A: Booting up system")
  timeout(1000, () => {
    console.log("A: Checking network connection")
    timeout(500, () => {
      console.log("A: Request complex computation")
      sendRequest(value => {
        console.log("A: Computation returned " + value)
        done()
      })
    })
  })

  console.log("B: Booting up system")
  timeout(100, () => {
    console.log("B: Server up and running")
    serverHandler = (callback) => {
      console.log("B: Starting heavy computation")
      timeout(2000, () => callback(42))
    }
    if (queueCallback) { serverHandler(queueCallback) }
  })

  function sendRequest(callback) {
    if (serverHandler) { serverHandler(callback) }
    else { queueCallback = callback }
  }
})

A: Booting up system
B: Booting up system
.B: Server up and running
..................A: Checking network connection
.........A: Request complex computation
B: Starting heavy computation
......................................A: Computation returned 42


In [40]:
function serverA(bootTime, done) {
  console.log("A: Booting up system...")
  timeout(bootTime, checkNetwork)

  function checkNetwork() {
    console.log("A: Checking network connection")
    timeout(500, sendRequest)
  }

  function sendRequest() {
    console.log("A: Request complex computation")
    sendNetworkRequest(callback)
  }

  function callback(value) {
    console.log("A: Computation returned " + value)
    done()
  }
}

In [41]:
function serverB(bootTime) {
  console.log("B: Booting up system...")
  timeout(bootTime, listenRequests)

  function listenRequests() {
    console.log("B: Server up and running")
    serverHandler = handler

    if (queueCallback) {
      serverHandler(queueCallback)
      queueCallback = null
    }
  }

  function handler(callback) {
    console.log("B: Starting heavy computation")
    timeout(2000, () => callback(42))
  }
}

In [9]:
function sendNetworkRequest(callback) {
  if(serverHandler) {
    serverHandler(callback)
  } else {
    queueCallback = callback
  }
}

var queueCallback = null
var serverHandler = null

progress((done) => {
  serverA(1000, done)
  serverB(100)
})

A: Booting up system...
B: Booting up system...
.B: Server up and running
.................A: Checking network connection
..........A: Request complex computation
B: Starting heavy computation
......................................A: Computation returned 42


# Wait, have you heard about promises?

A quick Promises explanation, according to the Promises/A+ specification, for people not familiar with the concept. We are going to explain how we return an object that will be resolved or will fail in the future, and how actions can be chained with the `.then()` method.

In [10]:
var { promisify } = require('util')
var delay = promisify(setTimeout)
var asyncProgress = fn => {
    var handle = setInterval(() => process.stdout.write('.'), 50)
    return new Promise((resolve, reject) => {
        fn().then(value => {
            clearTimeout(handle)
            resolve(value)
        }).catch(error => {
            clearTimeout(handle)
            reject(error)
        })
    })
}

asyncProgress(() => {
    return delay(1000)
        .then(() => {
            console.log('Hello world!')
            return "I will be back"
        })
        .then(value => delay(500).then(() => value))
})


..................Hello world!
..........

'I will be back'

In [11]:
function getServerA(bootTime) {
    console.log("A: Booting up system...")

    return Promise.delay(bootTime)
        .then(checkNetwork)
        .then(() => ({ sendRequest }))

    function checkNetwork() {
        console.log("A: Checking network connection")
        return delay(500)
    }

    function sendRequest(remoteServer) {
        console.log("A: Request complex computation")
        return remoteServer
            .compute(2)
            .then(value => {
                console.log
                console.log(`A: Computation returned ${value}`)
            })
    }
}

In [12]:
function getServerB(bootTime) {
    console.log("B: Booting up system...")

    return Promise.delay(bootTime).then(() => {
        console.log("B: Server up and running")
        return { compute }
    })

    function compute(value) {
        console.log("B: Starting heavy computation")
        return delay(2000).then(() => 40 + value)
    }
}

In [13]:
var Promise = require('bluebird')

asyncProgress(main)

function main () {
    return Promise.props({
        serverA: getServerA(1000),
        serverB: getServerB(100)
    }).then(({ serverA, serverB }) => {
        return serverA.sendRequest(serverB)
    })
}

A: Booting up system...
B: Booting up system...
.B: Server up and running
..................A: Checking network connection
.........A: Request complex computation
B: Starting heavy computation
......................................A: Computation returned 42


# Async/Await as an imperative, easier to read approach

Promises end up being a more functional programming approach, but a little more difficult to read for folks used to imperative programming. After function generators (aka coroutines) were introduced in the ES2015 specification, people started using it to replicate the Async/Await feature from C#, to be able to await Promises. We are going to see how it was used, and how it evolved to be officially adopted in the ES2017 specification.

We are also going to talk about Async/Await limitation of awaiting a single Promise at a time, and how other approaches might solve this problem.

In [14]:
async function getServerA(bootTime) {
    console.log("A: Booting up system...")
    await delay(bootTime)
    await checkNetwork()

    return { sendRequest }

    function checkNetwork() {
        console.log("A: Checking network connection")
        return delay(500)
    }

    async function sendRequest(remoteServer) {
        console.log("A: Request complex computation")
        var value = await remoteServer.compute(2)
        console.log(`A: Computation returned ${value}`)
    }
}

In [15]:
async function getServerB(bootTime) {
    console.log("B: Booting up system...")
    await delay(bootTime)
    console.log("B: Server up and running")
    return { compute }

    async function compute(value) {
        console.log("B: Starting heavy computation")
        await delay(2000)
        return 40 + value
    }
}

In [16]:
var { all } = Promise

asyncProgress(main)

async function main () {
    var [serverA, serverB] = await all([getServerA(), getServerB()])
    await serverA.sendRequest(serverB)
}

A: Booting up system...
B: Booting up system...
A: Checking network connection
B: Server up and running
.........A: Request complex computation
B: Starting heavy computation
......................................A: Computation returned 42


# Reactive Extensions, as in CSharp

After Reactive Extensions were introduced in C#, they were ported to several different programming environments, including JavaScript with Rx.js. So, we are going to explain the concept, showing a simple code example, and how it's possible to use the Rx.js API to manipulate and merge all sorts of different streams.

In [18]:
function getServerA(bootTime) {
    console.log("A: Booting up system...")

    return Promise.delay(bootTime)
        .then(checkNetwork)
        .then(() => ({ sendRequest }))

    function checkNetwork() {
        console.log("A: Checking network connection")
        return delay(500)
    }

    function sendRequest(remoteServer) {
        console.log("A: Request complex computation")

        return remoteServer.compute(2)
            .do(value => console.log(`A: Received value ${value}`))
            .take(10)
            .reduce((acc, value) => acc + value, 0)
            .toPromise()
            .then(total => console.log(`A: Computation returned ${total}`))
    }
}

In [19]:
function getServerB(bootTime) {
    console.log("B: Booting up system...")

    return Promise.delay(bootTime).then(() => {
        console.log("B: Server up and running")
        return { compute }
    })

    function compute(value) {
        console.log("B: Starting heavy computation")
        return Observable.interval(200)
            .scan((acc, x, i) => acc + i * 2, value)
    }
}

In [20]:
var { Observable } = require('rxjs')

asyncProgress(main)

function main () {
    return Promise.props({
        serverA: getServerA(1000),
        serverB: getServerB(100)
    }).then(({ serverA, serverB }) => {
        return serverA.sendRequest(serverB)
    })
}

A: Booting up system...
B: Booting up system...
.B: Server up and running
.................A: Checking network connection
..........A: Request complex computation
B: Starting heavy computation
....A: Received value 2
....A: Received value 4
...A: Received value 8
....A: Received value 14
....A: Received value 22
....A: Received value 32
....A: Received value 44
....A: Received value 58
....A: Received value 74
...A: Received value 92
A: Computation returned 350


# Channel-based programming, as in Go

Go leverages Communicating Sequential Processes (aka CSP), by using blocking channels to coordinate the communication of many goroutines. Even though JavaScript is single threaded, the library `js-csp` uses generators the same style of programming. We are going to show an example of how use it to manipulate streams of async events in an imperative manner. 

In [28]:
var { go, put, take, CLOSED, timeout, chan } = require('js-csp')
var toPromise = channel => {
    return new Promise(resolve =>
        go(function * () {
            resolve(yield take(channel))
        })
    )
}

In [29]:
async function getServerA(bootTime) {
    console.log("A: Booting up system...")
    await delay(bootTime)
    await checkNetwork()

    return { sendRequest }

    function checkNetwork() {
        console.log("A: Checking network connection")
        return delay(500)
    }

    async function sendRequest(remoteServer) {
        console.log("A: Request complex computation")
        var channel = remoteServer.compute(2)

        var value = await toPromise(csp.go(function * () {
            var acc = 0
            var count = 0
            while (true) {
                var value = yield take(channel)

                console.log(`A: Received ${value}`)

                acc += value
                count += 1

                if (count > 10) {
                    channel.close()
                    return acc
                }
            }
        }))

        console.log(`A: Computation returned ${value}`)
    }
}

In [30]:
async function getServerB(bootTime) {
    console.log("B: Booting up system...")
    await delay(bootTime)
    console.log("B: Server up and running")
    return { compute }

    function compute(value) {
        console.log("B: Starting heavy computation")

        var channel = csp.chan()

        go(function * () {
            var current = value
            var i = 1

            while (yield put(channel, current)) {
                current += i * 2
                i++

                yield timeout(200)
            }
        })

        return channel
    }
}

In [31]:
asyncProgress(main)

async function main () {
    var [serverA, serverB] = await all([getServerA(), getServerB()])
    await serverA.sendRequest(serverB)
}

A: Booting up system...
B: Booting up system...
A: Checking network connection
B: Server up and running
.........A: Request complex computation
B: Starting heavy computation
A: Received 2
....A: Received 4
....A: Received 8
...A: Received 14
....A: Received 22
....A: Received 32
....A: Received 44
....A: Received 58
....A: Received 74
...A: Received 92
....A: Received 112
A: Computation returned 462


In [25]:
var { delay } = require('bluebird')

async function * producer() {
  const stream = [1, 2, 3, 4, 5]
  for (let val of stream) {
    await delay(1000)
    yield val
  }
}

async function consumer () {
  for await (let val of producer()) {
    console.log(val)
  }
}

consumer()

SyntaxError: Unexpected token *

# Actor-based programming, as in Erlang and Elixir.

All of the previous async techniques don't solve JavaScripts limitation of running in a single thread. That can, however, be worked around using Actor based programming, just like in Erlang and Elixir. Actor based programming works with idea of Actors, which can be thought of asynchronous and independent objects. With that concept in mind, we use message passing to communicate with the actors. The big concept here is, because Actors are independent from each other, can run in different processes. Therefore, we can use a library like `comedy` to run our application across many machines or many processes within the same machine. We are going to demonstrate an example of how this can be accomplished.

# Comparison of all different methods used

```
+------------+-------------+------------------------------+
|            |  One Event  |        Multiple Events       |
+------------+-------------+------------------------------+
| Functional |  Callbacks  |      Reactive Extensions     |
|            |   Promises  |                              |
+------------+-------------+------------------------------+
| Imperative | Async/Await | Async Generators & Iterators |
|            |             |       Go-like channels       |
+------------+-------------+------------------------------+
```